In [1]:
import numpy as np
from keras import layers
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras.backend as K
from keras.layers import Dropout
#tf.K.set_image_data_format(‘channels_last’)
tf.keras.backend.set_image_data_format('channels_last')
#import matplotlib.pyplot as plt
#from matplotlib.pyplot import imshow
path = 'G:\\TEXT\\jupyter\\Skin_attempt 4\\base_dir\\train_dir'
train_datagen = ImageDataGenerator(rescale=1. / 255)
train = train_datagen.flow_from_directory(path, target_size=(227,227), class_mode='categorical')

Found 25587 images belonging to 7 classes.


In [2]:
type(train)

tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [3]:
type(train_datagen)

tensorflow.python.keras.preprocessing.image.ImageDataGenerator

In [4]:
print("Batch Size for Input Image : ",train[0][0].shape)
print("Batch Size for Output Image : ",train[0][1].shape)
print("Image Size of first image : ",train[0][0][0].shape)
print("Output of first image : ",train[0][1][0].shape)

Batch Size for Input Image :  (32, 227, 227, 3)
Batch Size for Output Image :  (32, 7)
Image Size of first image :  (227, 227, 3)
Output of first image :  (7,)


In [5]:
def AlexNet(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = BatchNormalization(axis = 3 , name = "bn0")(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
    X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = BatchNormalization(axis = 3 ,name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
    X = Flatten()(X)
    
    X = Dense(4096, activation = 'relu', name = "fc0")(X)
    
    #X = Dropout(0.3)(X)
    
    X = Dense(4096, activation = 'relu', name = 'fc1')(X)
    
    #X = Dropout(0.2)(X)
    
    X = Dense(7,activation='softmax',name = 'fc2')(X)
    
    model = Model(inputs = X_input, outputs = X, name='AlexNet')
    return model
alex = AlexNet(train[0][0].shape[1:])
alex.summary()

Model: "AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv0 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
bn0 (BatchNormalization)     (None, 55, 55, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max0 (MaxPooling2D)          (None, 27, 27, 96)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 27, 27, 256)       614656    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 27, 27, 256)       1024

In [6]:
path1 = 'G:\\TEXT\\jupyter\\Skin_attempt 4\\base_dir\\test_dir\\val_dir'
val_datagen = ImageDataGenerator(rescale=1. / 255)
val = val_datagen.flow_from_directory(path1, target_size=(227,227), class_mode='categorical')
type(val)
type(val_datagen)

Found 2003 images belonging to 7 classes.


tensorflow.python.keras.preprocessing.image.ImageDataGenerator

In [7]:
#1
#alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

opt = keras.optimizers.Adam(learning_rate=0.0001)
alex.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

alex.fit(train,epochs=5,validation_data=(val))

Epoch 1/5
800/800 [==============================] - 3407s 4s/step - loss: 1.1398 - accuracy: 0.5802 - val_loss: 0.8651 - val_accuracy: 0.7104
Epoch 2/5
800/800 [==============================] - 3693s 5s/step - loss: 0.7707 - accuracy: 0.7069 - val_loss: 1.0295 - val_accuracy: 0.6355
Epoch 3/5
800/800 [==============================] - 3317s 4s/step - loss: 0.6291 - accuracy: 0.7575 - val_loss: 0.8352 - val_accuracy: 0.7294
Epoch 4/5
800/800 [==============================] - 3413s 4s/step - loss: 0.5087 - accuracy: 0.8065 - val_loss: 0.8782 - val_accuracy: 0.6985
Epoch 5/5
800/800 [==============================] - 3349s 4s/step - loss: 0.4278 - accuracy: 0.8394 - val_loss: 0.9018 - val_accuracy: 0.7139


In [7]:
#2
learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,min_lr=0.00001)
alex.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
alex.fit(train,epochs=5,validation_data=val)

Epoch 1/5
800/800 [==============================] - 3471s 4s/step - loss: 2.0513 - accuracy: 0.3199 - val_loss: 1.2464 - val_accuracy: 0.5662
Epoch 2/5
800/800 [==============================] - 3656s 5s/step - loss: 1.2483 - accuracy: 0.5008 - val_loss: 1.2941 - val_accuracy: 0.5442
Epoch 3/5
800/800 [==============================] - 3388s 4s/step - loss: 1.0837 - accuracy: 0.5791 - val_loss: 0.9371 - val_accuracy: 0.6605
Epoch 4/5
800/800 [==============================] - 3637s 5s/step - loss: 0.9948 - accuracy: 0.6165 - val_loss: 1.1550 - val_accuracy: 0.6296
Epoch 5/5
800/800 [==============================] - 3507s 4s/step - loss: 0.8987 - accuracy: 0.6552 - val_loss: 1.4386 - val_accuracy: 0.5442


In [ ]:
#3
#alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

opt = keras.optimizers.Adam(learning_rate=0.001)
alex.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

alex.fit(train,epochs=5,validation_data=(val))

Epoch 1/5
800/800 [==============================] - 3630s 5s/step - loss: 2.2199 - accuracy: 0.3367 - val_loss: 1.4491 - val_accuracy: 0.5896
Epoch 2/5
800/800 [==============================] - 3343s 4s/step - loss: 1.4026 - accuracy: 0.4100 - val_loss: 1.4382 - val_accuracy: 0.5572
Epoch 3/5
216/800 [=======>......................] - ETA: 39:47 - loss: 1.3657 - accuracy: 0.4439

In [ ]:
#4
#alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

opt = keras.optimizers.Adam(learning_rate=0.01)
alex.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

alex.fit(train,epochs=5,validation_data=(val))

Epoch 1/5
800/800 [==============================] - 3309s 4s/step - loss: 17.5121 - accuracy: 0.2552 - val_loss: 1.5991 - val_accuracy: 0.2322
Epoch 2/5
350/800 [============>.................] - ETA: 31:22 - loss: 1.9522 - accuracy: 0.2591

In [11]:
#5
#alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

opt = keras.optimizers.Adam(learning_rate=0.0001)
alex.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

alex.fit(train,epochs=20,validation_data=(val))

Epoch 1/20
800/800 [==============================] - 3351s 4s/step - loss: 1.1365 - accuracy: 0.5780 - val_loss: 0.9809 - val_accuracy: 0.6745
Epoch 2/20
800/800 [==============================] - 35996s 45s/step - loss: 0.7808 - accuracy: 0.7039 - val_loss: 0.8333 - val_accuracy: 0.6975
Epoch 3/20
800/800 [==============================] - 3537s 4s/step - loss: 0.6194 - accuracy: 0.7620 - val_loss: 0.9364 - val_accuracy: 0.6825
Epoch 4/20
800/800 [==============================] - 3473s 4s/step - loss: 0.5128 - accuracy: 0.8050 - val_loss: 0.8799 - val_accuracy: 0.7209
Epoch 5/20
800/800 [==============================] - 6114s 8s/step - loss: 0.4332 - accuracy: 0.8360 - val_loss: 1.0122 - val_accuracy: 0.7174
Epoch 6/20
800/800 [==============================] - 3269s 4s/step - loss: 0.3608 - accuracy: 0.8630 - val_loss: 0.9464 - val_accuracy: 0.7119
Epoch 7/20
800/800 [==============================] - 4743s 6s/step - loss: 0.3069 - accuracy: 0.8838 - val_loss: 0.8581 - val_accurac

In [7]:
#6
#alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

opt = keras.optimizers.Adam(learning_rate=0.0001)
alex.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

alex.fit(train,batch_size=64,epochs=11,validation_data=(val))

Epoch 1/11
800/800 [==============================] - 3595s 4s/step - loss: 1.1557 - accuracy: 0.5788 - val_loss: 0.9906 - val_accuracy: 0.6435
Epoch 2/11
800/800 [==============================] - 3489s 4s/step - loss: 0.7689 - accuracy: 0.7086 - val_loss: 1.0432 - val_accuracy: 0.6355
Epoch 3/11
800/800 [==============================] - 3893s 5s/step - loss: 0.6138 - accuracy: 0.7662 - val_loss: 1.0861 - val_accuracy: 0.6925
Epoch 4/11
800/800 [==============================] - 3720s 5s/step - loss: 0.5093 - accuracy: 0.8067 - val_loss: 0.8187 - val_accuracy: 0.7044
Epoch 5/11
800/800 [==============================] - 3745s 5s/step - loss: 0.4261 - accuracy: 0.8394 - val_loss: 1.3373 - val_accuracy: 0.5652
Epoch 6/11
800/800 [==============================] - 3547s 4s/step - loss: 0.3602 - accuracy: 0.8619 - val_loss: 1.2755 - val_accuracy: 0.7029
Epoch 7/11
 34/800 [>.............................] - ETA: 50:58 - loss: 0.2944 - accuracy: 0.8842

KeyboardInterrupt: 

In [8]:
#7
opt = keras.optimizers.SGD(learning_rate=0.0001)
alex.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

alex.fit(train,epochs=5,validation_data=(val))

Epoch 1/5
800/800 [==============================] - 3263s 4s/step - loss: 1.4083 - accuracy: 0.4657 - val_loss: 1.2127 - val_accuracy: 0.5846
Epoch 2/5
800/800 [==============================] - 3248s 4s/step - loss: 1.1335 - accuracy: 0.5775 - val_loss: 1.0571 - val_accuracy: 0.6186
Epoch 3/5
800/800 [==============================] - 3249s 4s/step - loss: 1.0266 - accuracy: 0.6198 - val_loss: 0.9190 - val_accuracy: 0.6540
Epoch 4/5
800/800 [==============================] - 3217s 4s/step - loss: 0.9495 - accuracy: 0.6521 - val_loss: 1.0171 - val_accuracy: 0.6101
Epoch 5/5
800/800 [==============================] - 3375s 4s/step - loss: 0.8887 - accuracy: 0.6815 - val_loss: 0.8376 - val_accuracy: 0.6865


In [11]:
#7
opt = keras.optimizers.SGD(learning_rate=0.0001)
alex.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

alex.fit(train,epochs=5,validation_data=(val))

Epoch 1/5
800/800 [==============================] - 3296s 4s/step - loss: 0.8390 - accuracy: 0.6969 - val_loss: 0.8317 - val_accuracy: 0.6890
Epoch 2/5
800/800 [==============================] - 3242s 4s/step - loss: 0.8005 - accuracy: 0.7127 - val_loss: 0.8252 - val_accuracy: 0.6930
Epoch 3/5
800/800 [==============================] - 3259s 4s/step - loss: 0.7629 - accuracy: 0.7288 - val_loss: 0.9157 - val_accuracy: 0.6770
Epoch 4/5
165/800 [=====>........................] - ETA: 41:34 - loss: 0.7253 - accuracy: 0.7425

KeyboardInterrupt: 

In [ ]:
#8
opt = keras.optimizers.Adam(learning_rate=0.0001)
alex.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

hist=alex.fit(train,epochs=4,validation_data=(val))

In [7]:
#testing
path_test = 'G:\\TEXT\\jupyter\\Skin_attempt 4\\base_dir\\test_dir\\testing_dir'
test_datagen = ImageDataGenerator(rescale=1. / 255)
test = test_datagen.flow_from_directory(path_test, target_size=(227,227), class_mode='categorical')

Found 2003 images belonging to 7 classes.


In [9]:
#1
preds = alex.evaluate(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
alex.save("model3.h5")

63/63 [==============================] - 63s 1s/step - loss: 0.8435 - accuracy: 0.7244
Loss = 0.843549370765686
Test Accuracy = 0.7244133949279785


In [9]:
#2
preds = alex.evaluate(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
alex.save("model4.h5")

63/63 [==============================] - 68s 1s/step - loss: 1.4098 - accuracy: 0.5292
Loss = 1.4097552299499512
Test Accuracy = 0.5292062163352966


In [12]:
#5
preds = alex.evaluate(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
alex.save("model4.h5")

63/63 [==============================] - 65s 1s/step - loss: 1.7313 - accuracy: 0.7274
Loss = 1.7313017845153809
Test Accuracy = 0.7274088859558105


In [9]:
#6
preds = alex.evaluate(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
alex.save("model5.h5")

63/63 [==============================] - 51s 808ms/step - loss: 0.9292 - accuracy: 0.7214
Loss = 0.9291800856590271
Test Accuracy = 0.7214178442955017


In [12]:
#7
preds = alex.evaluate(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
alex.save("model6.h5")

63/63 [==============================] - 51s 803ms/step - loss: 0.8853 - accuracy: 0.6780
Loss = 0.8852553963661194
Test Accuracy = 0.6779830455780029


In [16]:
alex=keras.models.load_model("model4.h5")

In [9]:
#7
preds = alex.evaluate(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

63/63 [==============================] - 72s 1s/step - loss: 1.7313 - accuracy: 0.7274
Loss = 1.73130202293396
Test Accuracy = 0.7274088859558105


In [10]:
from matplotlib import pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [18]:
plt.plot(history.history['accuracy'])
plt.plot(alex.alex['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined